In [5]:
# !pip install -qU langchain-cli
# !pip install -qU langsmith
# !pip install -qU python-dotenv
# !pip install -qU langchain-openai
# !pip install -qU "langserve[all]"

In [6]:
import os

from dotenv import load_dotenv

load_dotenv()

# 读取LangSmith Key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "lang-env"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
GPT_GOD_API_KEY = os.getenv("GPT_GOD_API_KEY")


In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="glm-4-flash",
                   base_url="https://api.gptgod.online/v1/",
                   api_key=GPT_GOD_API_KEY
                   )

In [8]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-146c2311-5f31-4711-9685-304812e0db6d-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [9]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't have access to personal information about individuals, so I can't tell you your name. If you're looking for a way to find out your name, you would need to check official documents like your birth certificate, driver's license, or any other identification that you have. If you're referring to a different context or asking how to determine a name for something, please provide more details.", response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 12, 'total_tokens': 92}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-242e15c2-f51e-4693-a557-e268820c369a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 80, 'total_tokens': 92})

In [10]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content="Your name is Bob. Is there anything specific related to Bob you'd like to know or discuss?", response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 27, 'total_tokens': 47}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-36aeec81-1c7b-4bee-b75f-7405f56670eb-0', usage_metadata={'input_tokens': 27, 'output_tokens': 20, 'total_tokens': 47})

In [11]:
! pip install -qU langchain_community

In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    print(session_id)
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [22]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

Parent run 77eeb36f-0b80-4c73-9dc0-99a1462eacf6 not found for run 2722cf49-37c6-487b-8f20-1e626bef2ac5. Treating as a root run.


abc2


'Hello Bob! How can I assist you today? Do you have any questions or need information on a particular topic?'

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 38644498-ec48-4608-863f-b2086638cf6e not found for run b34852ee-ba49-4de7-ae4c-5908d091798a. Treating as a root run.


abc2


"Your name is Bob. Is there anything else you'd like to know or discuss, Bob?"

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [25]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello Bob! How can I assist you today?'

In [26]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [27]:
config = {"configurable": {"session_id": "abc5"}}

In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

Parent run 8a2984d0-305f-424c-b611-8b341f0dfa4d not found for run 6deb4db0-d7ce-48fe-bd6b-c70df64b7565. Treating as a root run.


abc5


'Hello Jim! How can I assist you today?'

In [29]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 844dc61c-f4f3-4903-96cd-a1819b5b414b not found for run 7012f266-e0ff-47aa-824c-49abd7af3ed9. Treating as a root run.


abc5


"Your name is Jim. Is there anything else you'd like to know or do?"